# using the Spanish representative feeders in PowerModelsDistribution.jl 
This is a notebook that demonstrates how to load the representative feeders of the Spanish dataset using PowerModelsDistribution.jl and perform basic operations.

In [37]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\mnumair\OneDrive - KU Leuven\PhD Agenda\2024\24-10 October\TSK-467 RepoRepFed\Representative-distribution-networks\RF_Spanish_representative_feeders\PMD`


In [38]:
using PowerModelsDistribution
using Ipopt
using FileIO

Here you can choose any of the 6 representative feeders of and load it as an ENGINEERING model of PowerModelsDistribution.jl using the `load()` with the file name as argument. below an example for loading the first representative feeder.

In [39]:
RF1 = load("Spanish_RF1.jld2")

Dict{String, Any} with 12 entries:
  "conductor_ids"  => [1, 2, 3, 4, 5]
  "bus"            => Dict{String, Any}("2189"=>Dict{String, Any}("lat"=>43.394…
  "name"           => "feeder_160_ntw_td004454"
  "settings"       => Dict{String, Any}("sbase_default"=>100000.0, "vbases_defa…
  "files"          => ["C:/Users/mnumair/OneDrive - KU Leuven/PhD Agenda/2024/2…
  "time_series"    => Dict{String, Any}("shape_3933"=>Dict{String, Any}("source…
  "voltage_source" => Dict{String, Any}("source"=>Dict{String, Any}("source_id"…
  "line"           => Dict{String, Any}("2189"=>Dict{String, Any}("length"=>23.…
  "data_model"     => ENGINEERING
  "shunt"          => Dict{String, Any}("line_loop.yngrnd"=>Dict{String, Any}("…
  "load"           => Dict{String, Any}("load8003"=>Dict{String, Any}("source_i…
  "linecode"       => Dict{String, Any}("310"=>Dict{String, Any}("b_fr"=>[0.001…

from the ENGINEERING model `RF1` it is possible to explore the network:
For example, lookup the linecode for the a line in the network:

In [40]:
line1_lc = RF1["line"]["2211"]["linecode"]

"217"

Then we can find the details for that linecode from the `linecode` dictionary:

In [41]:
RF1["linecode"][line1_lc] 

Dict{String, Any} with 6 entries:
  "b_fr" => [0.0014 -0.0003 -0.0003 -0.0003; -0.0003 0.0014 -0.0003 -0.0003; -0…
  "rs"   => [0.00134 0.0 0.0 0.0; 0.0 0.00134 0.0 0.0; 0.0 0.0 0.00134 0.0; 0.0…
  "xs"   => [9.7e-5 0.0 0.0 0.0; 0.0 9.7e-5 0.0 0.0; 0.0 0.0 9.7e-5 0.0; 0.0 0.…
  "b_to" => [0.0014 -0.0003 -0.0003 -0.0003; -0.0003 0.0014 -0.0003 -0.0003; -0…
  "g_to" => [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
  "g_fr" => [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]

and can see the formatted series resistance matrix for that linecode:

In [42]:
RF1["linecode"][line1_lc]["rs"]

4×4 Matrix{Float64}:
 0.00134  0.0      0.0      0.0
 0.0      0.00134  0.0      0.0
 0.0      0.0      0.00134  0.0
 0.0      0.0      0.0      0.0006579

However most of the interesting analysis happens using the MATHEMATICAL model which is used for optimization problems. So we have to transform `RF1` from the ENGINEERING data model to the MATHEMATICAL model `RF1_math` 

In [34]:
RF1_math = transform_data_model(RF1, kron_reduce = false, phase_project = false)

Dict{String, Any} with 18 entries:
  "is_kron_reduced" => false
  "conductor_ids"   => [1, 2, 3, 4, 5]
  "time_elapsed"    => 1.0
  "bus"             => Dict{String, Any}("32"=>Dict{String, Any}("vm_pair_lb"=>…
  "name"            => "feeder_160_ntw_td004454"
  "map"             => Dict{String, Any}[Dict("unmap_function"=>"_map_math2eng_…
  "settings"        => Dict{String, Any}("sbase_default"=>100000.0, "vbases_def…
  "gen"             => Dict{String, Any}("1"=>Dict{String, Any}("pg"=>[0.0, 0.0…
  "branch"          => Dict{String, Any}("32"=>Dict{String, Any}("br_r"=>[9.434…
  "storage"         => Dict{String, Any}()
  "switch"          => Dict{String, Any}()
  "is_projected"    => false
  "per_unit"        => true
  "data_model"      => MATHEMATICAL
  "shunt"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>"s…
  "transformer"     => Dict{String, Any}()
  "bus_lookup"      => Dict{Any, Int64}("2189"=>1, "2198_3"=>2, "2229_2"=>3, "2…
  "load"            => Dict{Stri

Then it is possible to perform an optimal power flow (OPF) using `PowerModelsDistribution.jl` functions

In [35]:
add_start_vrvi!(RF1_math)
RF1_opf = solve_mc_opf(RF1_math, IVRENPowerModel, Ipopt.Optimizer)

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    15128
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:     1198

Total number of variables............................:     4092
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints.................:     3276
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        3

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.99e-02 9.90e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.0709999
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 0.0375597
  "solution"           => Dict{String, Any}("branch"=>Dict{String, Any}("32"=>D…
  "objective_lb"       => -Inf

Then we can explore the results from the `"solution"` dictionary. FOr example we can find the voltage magnitude `vm` and angle `va` for the three phases of the first bus as follows

In [36]:
RF1_opf["solution"]["bus"]["1"]

Dict{String, Any} with 2 entries:
  "vi" => [-0.000511702, -0.850207, 0.852181, 0.2942]
  "vr" => [0.997854, -0.503239, -0.486422, -0.317697]

NOTE 1: The all of the above can be done for any of the representative feeders of the dataset, bus simply swapping the file path in the `load()` function.

NOTE 2: This is just an example of how to load the Spanish representative feeders in PowerModelsDistribution.jl. For more information on how to use PowerModelsDistribution.jl package, please refer to the [documentation](https://lanl-ansi.github.io/PowerModelsDistribution.jl/stable/).
